In [1]:
# dataops src
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import yfinance as yf

import os
from datetime import datetime

# etl src
import numpy as np
import pandas as pd
import os
from datetime import datetime

import tensorflow as tf
from tensorflow.keras import layers

# model src
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import os
from datetime import datetime
from sklearn.preprocessing import minmax_scale

import yfinance as yf

import torch
import torch.nn as nn

# evaluation src
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime

# prediction src
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime

# --------------------------------

# etl univariate scripts
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import os
from datetime import datetime
from sklearn.preprocessing import minmax_scale
import yfinance as yf

# etl multivariate scripts
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import os
from datetime import datetime
from sklearn.preprocessing import minmax_scale
import yfinance as yf

# multivariate current day pred scripts
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import os
from datetime import datetime
from sklearn.preprocessing import minmax_scale
import yfinance as yf

# multivariate full pipeline
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import os
from datetime import datetime
from sklearn.preprocessing import minmax_scale
import yfinance as yf

# multivariate train pred
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import os
from datetime import datetime
from sklearn.preprocessing import minmax_scale
import yfinance as yf

# SRC

## Assets

In [2]:
import pandas as pd

# Global variables for window and horizon size
WINDOW_SIZE_WEEK = 7
WINDOW_SIZE_MONTH = 30
WINDOW_SIZE_2_MONTH = 60

HORIZON_DAY = 1
HORIZON_WEEK = 7
HORIZON_MONTH = 30

# Timesteps into the future
INTO_FUTURE_1_DAY = 1
INTO_FUTURE_WEEK = 7
INTO_FUTURE_2_WEEK = 14
INTO_FUTURE_MONTH = 30
INTO_FUTURE_2_MONTH = 60

# Block reward values
BLOCK_REWARD_1 = 50 # After 3 January 2009 (2009-01-03) - this block reward isn't accounted for in BTC prices
BLOCK_REWARD_2 = 25 # After 28 November 2012
BLOCK_REWARD_3 = 12.5 # After 9 July 2016
BLOCK_REWARD_4 = 6.25 # After 11 May 2020
BLOCK_REWARD_5 = 3.125 # After 20 April 2024

# Block reward dates (datetime form of the above date stamps for block rewards)
BLOCK_REWARD_1_DATETIME = pd.Timestamp('2009-01-03')
BLOCK_REWARD_2_DATETIME = pd.Timestamp('2012-11-28')
BLOCK_REWARD_3_DATETIME = pd.Timestamp('2016-07-09')
BLOCK_REWARD_4_DATETIME = pd.Timestamp('2020-05-11')
BLOCK_REWARD_5_DATETIME = pd.Timestamp('2024-04-20')

# Batch sizes
NBEATS_BATCH_SIZE = 1024
BATCH_SIZE = 1024

# N-BEATS hyperparameters
# Values from N-BEATS paper in Figure 1 and Table 18/Appendix D
NBEATS_N_EPOCHS = 100
NBEATS_N_NEURONS = 512
NBEATS_N_LAYERS = 4
NBEATS_N_STACKS = 30

NBEATS_INPUT_SIZE = WINDOW_SIZE_WEEK * HORIZON_DAY # Lookback period in Appendix D
NBEATS_THETA_SIZE = NBEATS_INPUT_SIZE + HORIZON_DAY

# Ensemble models training hyperparameters
ENSEMBLE_NUM_ITER = 10
ENSEMBLE_NUM_ITER_TEST = 1
ENSEMBLE_NUM_EPOCHS = 300
ENSEMBLE_NUM_EPOCHS_TEST = 1

## DataOps

In [3]:
# Live training dataset loading

# Function to generate historical training dataset
def get_historical_btc_data(period_years=10, interval_days=1):
  """
  Returns historical BTC data using a period in years and interval in days
  Parameters
  ----------
  period_years: Period in years
  interval_days: Interval in days
  """
  df_btc_price = yf.download(tickers='BTC-USD', period=f'{period_years}y', interval=f'{interval_days}d')
  print(f"Head of BTC prices: \n{df_btc_price.head()}")
  print(f"BTC prices info: \n{df_btc_price.info()}")

  # Only looking for the closing price for each record
  df_btc_price_closing = pd.DataFrame(df_btc_price['Close']).rename(columns={'Close': 'Price'})
  print(df_btc_price_closing.head())

  return df_btc_price_closing

# Function to generate future dates for prediction data
def get_future_dates(start_date, into_future, offset=1):
    """
    Returns array of datetime values ranging from start_date to start_date +
    into_future (horizon).

    Parameters
    ----------
    start_date: date to start range (np.datetime64).
    into_future: number of days to add onto start date for range (int).
    offset: number of days to offset start_date by (default = 1).
    """
    start_date = start_date + np.timedelta64(offset, "D")
    end_date = start_date + np.timedelta64(into_future, "D")

    return np.arange(start_date, end_date, dtype="datetime64[D]")

## ETL

In [4]:
# Function to include block reward to training and testing dataset before windowing is in effect
def include_block_reward(df_btc_price):
    """
    Includes block reward to an existing dataframe containing BTC prices with
    datetime indexes. Should be used before windowing of dataset is in effect.

    Parameters
    ----------
    df_btc_price: DataFrame containing BTC prices with datetime indexes.
    """
    # TODO: Need to change this function when block reward changes in the future and also need to better manage block reward for the future

    block_reward_1_days = (BLOCK_REWARD_2_DATETIME - df_btc_price.index[0]).days
    block_reward_2_days = (BLOCK_REWARD_3_DATETIME - df_btc_price.index[0]).days
    block_reward_3_days = (BLOCK_REWARD_4_DATETIME - df_btc_price.index[0]).days
    block_reward_4_days = (df_btc_price.index[-1] - BLOCK_REWARD_4_DATETIME).days

    print(f"Days from block reward 1: {block_reward_1_days}\n",
          f"Days from block reward 2: {block_reward_2_days}\n",
          f"Days from block reward 3: {block_reward_3_days}\n",
          f"Days from block reward 4: {block_reward_4_days}\n")

    # Adding the block_reward column
    df_btc_price_block_reward = df_btc_price.copy()
    df_btc_price_block_reward["block_reward"] = None

    # Set values of block_reward column (it's the last column here hence -1 indexing using iloc)
    df_btc_price_block_reward.iloc[:block_reward_2_days, -1] = BLOCK_REWARD_2
    df_btc_price_block_reward.iloc[block_reward_2_days:block_reward_3_days, -1] = BLOCK_REWARD_3
    df_btc_price_block_reward.iloc[block_reward_3_days:, -1] = BLOCK_REWARD_4

    print("Head of dataframe with block reward included: \n", df_btc_price_block_reward.head())
    print("Tail of dataframe with block reward included: \n", df_btc_price_block_reward.tail())

    return df_btc_price_block_reward

def get_labelled_windows(x, horizon=HORIZON_DAY):
    """
    Create labels for windowed dataset.

    E.g. if horizon is 1, then:
    x: [0, 1, 2, 3, 4, 5, 6, 7] -> output: ([0, 1, 2, 3, 4, 5, 6], [7])
    """
    return x[:, :-horizon], x[:, -horizon:]

# View numpy arrays as windows
def make_windows(x, window_size=WINDOW_SIZE_WEEK, horizon=HORIZON_DAY):
    """
    Turns a 1D array into a 2D array of sequential labelled windows of
    window_size with horizon size labels.

    Returns both a 2D array containing full windowed X values with shape
    (number of samples, window size), and a 2D array containing full
    labelled y values with shape (number of samples, horizon size).
    """
    # TODO: In the future, function could be implemented using https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/timeseries_dataset_from_array

    # 1. Create a window of specific window_size (add the horizon on the end for labelling later)
    window_step = np.expand_dims(np.arange(window_size + horizon), axis=0)

    # 2. Create a 2D of multiple window steps (minus 1 to account for 0 indexing)
    window_indexes = window_step + np.expand_dims(np.arange(len(x) - (window_size + horizon - 1)), axis=0).T # Create 2D array of windows of window size
    # print(f"Window indexes: \n {window_indexes, window_indexes.shape}")

    windowed_array = x[window_indexes]
    # print(windowed_array)

    # 4. Get the labelled window
    windows, labels = get_labelled_windows(windowed_array, horizon=horizon)

    return windows, labels

# Windowing function for block reward (multivariate dataset)
def make_windows_labels_multivariate(df_btc_price_block_reward, window=WINDOW_SIZE_WEEK):
    """
    Returns windows and labels using a dataframe containing both BTC prices and
    block reward.

    Parameters
    ----------
    df_btc_price_block_reward: DataFrame containing BTC prices and block reward.
    window: Window size.
    """
    # Make a copy of the BTC historical data with block reward feature
    df_btc_price_windowed = df_btc_price_block_reward.copy()

    # Add windowed columns
    for i in range(window):
        df_btc_price_windowed[f"Price{i + 1}"] = df_btc_price_windowed["Price"].shift(periods=i + 1)

    print("Head of windowed dataframe: \n", df_btc_price_windowed.head(10))

    # Create X & y, remove the NaNs and convert to float32 to prevent tensorflow errors
    X = df_btc_price_windowed.dropna().drop('Price', axis=1).astype(np.float32)
    y = df_btc_price_windowed.dropna()['Price'].astype(np.float32)

    print("Head of X: \n", X.head())
    print("Head of y: \n", y.head())
    print(f"Shape of X: {X.shape} \n", f"Shape of y: {y.shape}")

    return X, y

# Implementation of train/test split after windowing is in effect
def make_train_test_splits(windows, labels, test_split=0.2):
    """
    Splits matching pairs of windows and labels into train and test splits.
    """
    split_size = int(len(windows) * (1 - test_split)) # This will default to 80% train and 20% test
    train_windows = windows[:split_size]
    train_labels = labels[:split_size]
    test_windows = windows[split_size:]
    test_labels = labels[split_size:]

    print(train_windows.shape, test_windows.shape, train_labels.shape, test_labels.shape)

    return train_windows, test_windows, train_labels, test_labels

# Function to generate train and test datasets using the tf.data API
def gen_train_test_datasets(X_train, y_train, X_test, y_test, batch_size=BATCH_SIZE):
    """
    Returns the train and test datasets using the tf.data API from train and
    test arrays.

    Parameters
    ----------
    X_train: Training windows.
    y_train: Training labels.
    X_test: Testing windows.
    y_test: Testing labels.
    batch_size: Batch size used for training.
    """
    # 1. Turn train and test arrays into tensor Datasets
    train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
    train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

    test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
    test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

    # 2. Combine features & labels
    train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
    test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

    # 3. Batch and prefetch for optimal performance
    train_dataset = train_dataset.batch(batch_size=batch_size).prefetch(tf.data.AUTOTUNE)
    test_dataset = test_dataset.batch(batch_size=batch_size).prefetch(tf.data.AUTOTUNE)

    return train_dataset, test_dataset

def save_train_test_data(train_windowed, train_labels, test_windowed, test_labels, features, save_path=None):
    """
    Saves the train and test datasets to save_path.
    """
    now = datetime.now()
    now_date_time = now.strftime('%d_%m_%Y_%H_%M_%S')

    df_train_windows = pd.DataFrame(train_windowed, columns=features)
    df_train_labels = pd.DataFrame(train_labels)
    df_test_windows = pd.DataFrame(test_windowed, columns=features)
    df_test_labels = pd.DataFrame(test_labels)

    if save_path == None:
        df_train_windows.to_csv(path_or_buf=os.path.join("train_windowed" + "_" + now_date_time + ".csv"))
        df_train_labels.to_csv(path_or_buf=os.path.join("train_labels" + "_" + now_date_time + ".csv"))
        df_test_windows.to_csv(path_or_buf=os.path.join("test_windowed" + "_" + now_date_time + ".csv"))
        df_test_labels.to_csv(path_or_buf=os.path.join("test_labels" + "_" + now_date_time + ".csv"))
    else:
        df_train_windows.to_csv(path_or_buf=os.path.join(save_path, "train_windowed" + "_" + now_date_time + ".csv"))
        df_train_labels.to_csv(path_or_buf=os.path.join(save_path, "train_labels" + "_" + now_date_time + ".csv"))
        df_test_windows.to_csv(path_or_buf=os.path.join(save_path, "test_windowed" + "_" + now_date_time + ".csv"))
        df_test_labels.to_csv(path_or_buf=os.path.join(save_path, "test_labels" + "_" + now_date_time + ".csv"))

## Model

In [5]:
## Modelling checkpoint
def create_model_checkpoint(model_name, save_path="btc_predict_model"):
    """
    Uses the model_name passed and saves model and weights to save_path using
    datetime.now().
    """
    now = datetime.now()
    now_date_time = now.strftime('%d_%m_%Y_%H_%M_%S')
    return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name + "_" + now_date_time),
                                              verbose=1, save_best_only=True)


class ARModel(tf.keras.Model):
    def __init__(self, shape):
        """
        Initializes Autoregressive model using below parameters.

        Parameters
        ----------
        shape: Tuple of (N, M) where N is the number of data points
        and M is the window size.
        model_name: Model's name.
        """
        super(ARModel, self).__init__()
        in_shape = shape[0]
        out_shape = shape[1]
        w_init = tf.random.normal(shape, mean=0, stddev=0.01, dtype='float32')
        self.w = tf.Variable(w_init, name='ar_w')

    def call(self, input_X, training=False):
        return tf.matmul(input_X, self.w) + tf.reduce_sum(self.w, axis=0)

def get_AR_model(X_shape, y_shape):
    layer_size = 30
    input_0 = tf.keras.Input(shape=X_shape)
    output_0 = ARModel((X_shape, layer_size))(input_0)
    output_1 = ARModel((layer_size, y_shape))(output_0)
    AR_model = tf.keras.Model(input_0, output_1)

    return AR_model

# Create N-BEATS custom layer
class NBEATSBlock(tf.keras.layers.Layer):
    def __init__(self, input_size: int, theta_size: int, horizon: int, n_neurons: int, n_layers: int, **kwargs):
        """
        Initializes NEATSBlock class using below parameters.

        Parameters
        ----------
        input_size: Size of backcast (window size).
        theta_size: Number of neurons in output dense layer, backcast + forecast.
        horizon: Size of horizon, forecast prediction.
        n_neurons: Number of neurons in fully connected dense layer.
        n_layers: Number of fully connected dense layers.
        kwargs: Passed to tf.keras.layers.Layers
        """
        super().__init__(**kwargs)
        self.input_size = input_size
        self.theta_size = theta_size
        self.horizon = horizon
        self.n_neurons = n_neurons
        self.n_layers = n_layers

        # One block contains a stack of 4 fully connected dense layers, each has a ReLU activation function
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for layer in range(n_layers)]

        # Output of block is a dense layer of theta_size with linear activation
        self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear")

    def __call__(self, inputs):
        """
        Runs when the layer is called.
        """
        x = inputs
        for layer in self.hidden: # pass inputs through each hidden layer
            x = layer(x)
        theta = self.theta_layer(x)

        # Output the backcast and forecast from theta
        backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]

        return backcast, forecast

def get_NBEATS_model(model_name="NBEATS_model"):
        """
        Returns a NBEATS model using the Functional API.

        Parameters
        ----------
        model_name: Model's name.
        """
        # TODO: Adjust this function to take into consideration if more features are used for the model, if more features are
        # used, then the NBEATS_INPUT_SIZE + 1 will need to change.
        # 1. Setup NBEATSBlock layer
        nbeats_block_layer = NBEATSBlock(input_size=NBEATS_INPUT_SIZE + 1, theta_size=NBEATS_THETA_SIZE, horizon=HORIZON_DAY,
                                         n_neurons=NBEATS_N_NEURONS, n_layers=NBEATS_N_LAYERS, name="initial_block")

        # 2. Create input to stacks
        stack_input = layers.Input(shape=(NBEATS_INPUT_SIZE + 1), name="stack_input")

        # 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
        backcast, forecast = nbeats_block_layer(stack_input)
        # Add in subtraction residual connections
        residuals = layers.subtract([stack_input, backcast], name=f"subtract_00")

        # 4. Create stacks of blocks
        for i, _ in enumerate(range(NBEATS_N_STACKS - 1)):
            # First stack is already created in # 3
            # 5. Use NBEATSBlock to calculate the backcast as well as block's forecast
            backcast, block_forecast = NBEATSBlock(
                input_size=NBEATS_INPUT_SIZE + 1,
                theta_size=NBEATS_THETA_SIZE,
                horizon=HORIZON_DAY,
                n_neurons=NBEATS_N_NEURONS,
                n_layers=NBEATS_N_LAYERS,
                name=f"NBEATSBlock_{i}"
            )(residuals) # Pass in the residuals (the backcast)

            # 6. Create the double residual stacking
            residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
            forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

        # 7. Put the stack model together
        NBEATS_model = tf.keras.Model(inputs=stack_input, outputs=forecast, name=model_name)

        return NBEATS_model

# Implement a layer normalization to normalize across the features dimension
# Also helps with unstable gradients
class LNRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        """
        Initializes a Layer Normalization RNN cell using below parameters.

        Parameters
        ----------
        units: Dimensionality of the output space of the SimpleRNNCell.
        activation: Activation function to use.
        """
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.rnn_cell = layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.activation = tf.keras.activations.get(activation)

    def __call__(self, inputs, states, training=True):
        """
        Runs when layer is called.
        """
        outputs, new_states = self.rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))

        return norm_outputs, [norm_outputs]

def get_LNRNN_model(window=WINDOW_SIZE_WEEK, horizon=HORIZON_DAY, model_name="LNRNN_model"):
    """
    Returns a LNRNN model using the LNRNNCell implementation, with the
    Sequential API.

    Parameters
    ----------
    window: Window size.
    horizon: Horizon size.
    model_name: Model's name.
    """
    # TODO: Adjust this function to take into consideration if more features are used for the model, if more features are
    # used, then the window + 1 will need to change.
    LNRNN_model = tf.keras.Sequential([
        layers.Lambda(lambda x: tf.expand_dims(x, axis=1)),
        layers.RNN(LNRNNCell(units=horizon), return_sequences=False, input_shape=[None, window + 1]),
        layers.Dense(horizon)
    ], name=model_name)

    return LNRNN_model

class LSTMModel():
    def __init__(self, window=WINDOW_SIZE_WEEK, br=True, horizon=HORIZON_DAY, n_lstm_neurons=128):
        """
        Initializes a LSTMModel class using below parameters.

        Parameters
        ----------
        window: Window size.
        horizon: Horizon size.
        n_lstm_neurons: Number of neurons to be used in LSTM layer.
        """
        # TODO: Adjust this function to take into consideration if more features are used for the model, if more features are
        # used, then the window + 1 will need to change.
        if br == True:
            self.inputs_layer = layers.Input(shape=(window + 1))
        else:
            self.inputs_layer = layers.Input(shape=(window))
        self.lambda_layer = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))
        self.lstm_layer = layers.LSTM(n_lstm_neurons, activation="relu")
        self.outputs_layer = layers.Dense(horizon)

    def get_model(self, model_name="LSTM_model"):
        """
        Returns a LSTM model using the Functional API, utilizing the
        inputs_layer and outputs_layer.

        Parameters
        ----------
        model_name: Model's name.
        """
        inputs = self.inputs_layer
        x = self.lambda_layer(inputs)
        x = self.lstm_layer(x)
        outputs = self.outputs_layer(x)

        lstm_model = tf.keras.Model(inputs=inputs, outputs=outputs, name=model_name)

        return lstm_model

class DenseModel():
    def __init__(self, n_neurons=128, horizon=HORIZON_DAY):
        """
        Initializes a DenseModel class using below parameters.

        Parameters
        ----------
        n_neurons: Number of neurons in dense layers.
        horizon: Horizon size.
        """
        self.n_neurons = n_neurons
        self.horizon = horizon

    def get_model(self, model_name="dense_model"):
        """
        Returns the dense model, using the Sequential API.

        Parameters
        ----------
        model_name: Model's name
        """
        dense_model = tf.keras.Sequential([
            layers.Dense(self.n_neurons, activation="relu"),
            layers.Dense(self.horizon, activation="relu")
        ], name=model_name)

        return dense_model

class WaveNet():
    def __init__(self):
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(input_shape=[None, 1]))

        for rate in (1, 2, 4, 8) * 2:
            # Define a spacing between the values in a kernel
            # 3x3 will be similar to 5x5 with a spacing of 2
            self.model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal", activation="relu", dilation_rate=rate))

        self.model.add(keras.layers.Conv1D(filters=10, kernel_size=1))

    def get_model(self):
        return self.model

# Generate an ensemble of models using various loss functions
def get_ensemble_models(models, train_data, test_data, horizon=HORIZON_DAY, num_iter=10, num_epochs=100,
                        loss_funcs=["mae", "mse", "mape"]):
    """
    Returns a list of num_iter models each trained on MAE, MSE and MAPE loss
    functions by default.

    For instance, if num_iter = 10, a list of 30 trained models will be returned.
    10 * len(loss_funcs) * 3 = 90
    """
    # Make empty list for trained ensemble models
    ensemble_models = []

    # Create num_iter number of models per loss function for NBeats, LSTM and Dense models
    for i in range(num_iter):
        # Build and fit ensemble of models
        for model in models:
            for loss_func in loss_funcs:
                print(f"Optimizing model by reducing: {loss_func} for epochs: {num_epochs}, num_iter: {i}, model: {model.name}")

                # Compile model with current loss function
                model.compile(loss=loss_func, optimizer=tf.keras.optimizers.Adam(), metrics=["mae", "mse"])

                # Fit the model
                model.fit(train_data, epochs=num_epochs, verbose=2, validation_data=test_data,
                          callbacks=[#create_model_checkpoint(model_name=f"{model.name}_{i}_{loss_func}"),
                                     tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                                     tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])

                # Append fitted model to list of ensemble models
                ensemble_models.append(model)

    return ensemble_models

def make_ensemble_preds(ensemble_models, input_data):
    """
    Returns predictions of ensemble models.

    Parameters
    ----------
    ensemble_models: Trained ensemble of models.
    input_data: Data to be predicted with.
    """
    ensemble_preds = []

    for model in ensemble_models:
        preds = model.predict(input_data) # Make predictions with current ensemble model
        ensemble_preds.append(preds)

    return tf.constant(tf.squeeze(ensemble_preds))

def make_preds(model, input_data):
    """
    Uses model to make predictions on input_data.

    Parameters
    ----------
    model : trained model
    input_data : windowed input data (same kind of data model was trained on)

    Returns model predictions on input_data.
    """
    forecast = model.predict(input_data)
    return tf.squeeze(forecast) # return 1D array of predictions

def get_ensemble_models_summary(models):
    """
    Generates model summaries of ensemble models.

    Parameters
    ----------
    models: Ensemble of models.
    """
    for model in models:
        print(model.summary())

def auto_regressive_model(window_size=WINDOW_SIZE_WEEK, lr=0.1):
    model = nn.Linear(window_size, 1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, criterion, optimizer

class RNNModel_torch(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_rnnlayers, n_outputs):
        super(RNNModel_torch, self).__init__()
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_outputs = n_outputs
        self.n_rnnlayers = n_rnnlayers

        self.rnn = nn.RNN(
                input_size=self.n_inputs,
                hidden_size=self.n_hidden,
                num_layers=self.n_rnnlayers,
                nonlinearity="relu",
                batch_first=True)

        self.fc = nn.Linear(self.n_hidden, self.n_outputs)

    def forward(self, inputs):
        hidden_state = torch.zeros(self.n_rnnlayers, inputs.size(0), self.n_hidden).to(device)

        out, time_step_val = self.rnn(inputs, hidden_state)
        out = self.fc(out[:, -1, :])

        return out

class LSTMModel_torch(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_rnnlayers, n_outputs):
        super(LSTMModel_torch, self).__init__()
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_outputs = n_outputs
        self.n_rnnlayers = n_rnnlayers

        self.lstm = nn.LSTM(input_size=self.n_inputs,
                          hidden_size=self.n_hidden,
                          num_layers=self.n_rnnlayers,
                          batch_first=True)

        self.fc = nn.Linear(self.n_hidden, self.n_outputs)

    def forward(self, inputs):
        hidden_state = torch.zeros(self.n_rnnlayers, inputs.size(0), self.n_hidden).to(device)
        cell_state = torch.zeros(self.n_rnnlayers, inputs.size(0), self.n_hidden).to(device)

        out, time_step_val = self.lstm(inputs, (hidden_state, cell_state))
        out = self.fc(out[:, -1, :])

        return out

def BGD_training(model, criterion, optimizer, train_windows, train_labels, test_windows, test_labels, epochs=200):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)

    for epoch in range(epochs):
        optimizer.zero_grad()

        outputs = model(train_windows)
        loss = criterion(outputs, train_labels)

        loss.backward()
        optimizer.step()

        train_losses[epoch] = loss.item()

        test_outputs = model(test_windows)
        test_loss = criterion(test_outputs, test_labels)

        test_losses[epoch] = test_loss.item()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch + 1} / {epochs}, test loss: {test_loss.item()}")

    return train_losses, test_losses

## Evaluation

In [6]:
def mean_absolute_scaled_error(y_true, y_pred):
    """
    Implement MASE (assuming to seasonality of data). MASE measures the
    forecast error compared to the error of a naive forecast. When he have a
    MASE = 1, that means the model is exactly as good as just picking the last
    observation. An MASE = 0.5, means that our model has doubled the prediction
    accuracy.
    """
    mae = tf.reduce_mean(tf.abs(y_true - y_pred))

    # Find MAE of naive forecast (no seasonality)
    mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # the seasonality is 1 day (hence the shifting of 1 day)

    return mae / mae_naive_no_season

# Updated evaluate_preds for large horizons (greater than 1 day)
def evaluate_preds(y_true, y_pred):
    """
    Evaluate mae, mse, rmse, mape, mase metrics for predictions. Updated
    for large horizons as well (greater than 1 day)
    """
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # Calculate various metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # MSE gives emphasis on outliers, as they're squared
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)

    # Account for different sized horizons, if the horizon is greater than 1 day, reduce it to a single number using the mean
    if mae.ndim > 0:
        mae = tf.reduce_mean(mae)
        mse = tf.reduce_mean(mse)
        rmse = tf.reduce_mean(rmse)
        mape = tf.reduce_mean(mape)
        mase = tf.reduce_mean(mase)

    return {"mae": mae.numpy(),
            "mse": mse.numpy(),
            "rmse": rmse.numpy(),
            "mape": mape.numpy(),
            "mase": mase.numpy()}

# Create a function to take in model predictions and truth values and return evaluation metrics
def evaluate_preds_1_day_horizon(y_true, y_pred):
    """
    Evaluate mae, mse, rmse, mape, mase metrics for predictions.
    """
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_true = tf.cast(y_pred, dtype=tf.float32)

    # Calculate various metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)

    return {"mae": mae.numpy(),
            "mse": mse.numpy(),
            "rmse": rmse.numpy(),
            "mape": mape.numpy(),
            "mase": mase.numpy()}

# Comparing the performance of models using different metrics
def compare_model_metrics(model_results: list, model_names: list, metrics=['mae', 'mse', 'rmse', 'mape', 'mase']):
    """
    Plots comparison of metrics for models.

    Parameters
    ----------
    models : List of dictionaries containing model metrics.
    i.e. model_results=[model_1_results, model_2_results]. Where
    model_1_results and model_2_results are dictionaries of metrics.

    model_names : List of model names.

    metrics : Metrics for comparison. Default is ['mae', 'mse', 'rmse', 'mape',
    'mase'].
    """
    for metric in metrics:
        pd.DataFrame({model_name: model_result[metric] for (model_name, model_result) in zip(model_names, model_results)},
                     index=[metric]).plot(figsize=(10, 7), kind='bar')

## Prediction

In [7]:
def get_upper_lower_confidence(preds):
    """
    Returns the upper and lower bounds of the 95% confidence level using the
    following logic:

    95% confidence upper, lower bounds = mean of preds +/- (standard deviation
    of preds * 1.96)

    Parameters
    ----------
    preds: Predictions of ensemble of models.
    """
    # 1. Take the predictions of multiple randomly initialized deep learning neural networks using the preds parameter
    # 2. Measure the standard deviation of the predictions
    std = tf.math.reduce_std(preds, axis=0)

    # 3. Multiply the standard deviation by 1.96
    interval = 1.96 * std

    # 4. Get the prediction interval's upper and lower bounds
    preds_mean = tf.reduce_mean(preds, axis=0)
    lower, upper = preds_mean - interval, preds_mean + interval

    return lower, upper

# Function to make predictions into future (can include retraining the model with the predicted data appended, everytime the model makes a prediction)
# 1. Create function to make predictions into the future
def make_future_forecast(models, values, into_future, window_size=WINDOW_SIZE_WEEK) -> list:
    """
    Makes future forecasts into_future steps after values ends.

    Returns future forecasts as list of floats.

    Parameters
    ----------
    values: BTC price labels after windowing is in effect.
    models: List of ensemble models to predict forecast using.
    into_future: How many days into future to make forecast for.
    window_size: Window size.
    """
    # 2. Make an empty list for future forecasts/prepare data to forecast on
    future_forecast = []
    last_window = values[-window_size:] # Only want preds from the last window (this will get updated after every prediction)

    # 3. Make into_future number of predictions, altering the data which gets predicted on each time
    for day in range(into_future):
        # Predict on last window then append it again and again (model will start to make forecasts on its own forecasts)
        ensemble_preds = make_ensemble_preds(ensemble_models=models, input_data=tf.expand_dims(last_window, axis=0))
        future_pred = np.median(ensemble_preds, axis=0)
        print(f"Predicting on: \n {last_window} -> Prediction: {tf.squeeze(future_pred).numpy()}\n")

        # Append predictions on future_forecast
        future_forecast.append(tf.squeeze(future_pred).numpy())

        # Update last window with the new pred and get window_size of most recent preds (model was trained on window_size windows)
        last_window = np.append(last_window, future_pred)[-window_size:]

    return future_forecast

# Scripts

## ETL

### Univariate

In [8]:
df_btc_price_closing = get_historical_btc_data()

# Take the full dataset's timesteps and closing prices
dataset_timesteps = df_btc_price_closing.index.to_numpy()
dataset_prices = df_btc_price_closing['Price'].to_numpy()

# Testing out the windowing function for multiple records
dataset_full_windows, dataset_full_labels = make_windows(dataset_prices, window_size=WINDOW_SIZE_WEEK, horizon=HORIZON_DAY)
print(dataset_full_windows.shape, dataset_full_labels.shape)

print(pd.DataFrame(dataset_prices))
print(pd.DataFrame(dataset_full_windows))
print(pd.DataFrame(dataset_full_labels))

# Testing out the train/test split function
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(dataset_full_windows, dataset_full_labels)
print(train_windows.shape, test_windows.shape, train_labels.shape, test_labels.shape)

[*********************100%%**********************]  1 of 1 completed

Head of BTC prices: 
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

              Volume  
Date                  
2014-09-17  21056800  
2014-09-18  34483200  
2014-09-19  37919700  
2014-09-20  36863600  
2014-09-21  26580100  
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3549 entries, 2014-09-17 to 2024-06-04
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3549 non-null   float64
 1   High       3549 non-null   float64
 2   Low       

### Multivariate

In [9]:
df_btc_price_closing = get_historical_btc_data()

# Including block reward to the full BTC closing price dataset
df_btc_price_block_reward = include_block_reward(df_btc_price=df_btc_price_closing)

# Obtain the full windows and labels with block reward included
dataset_full_windows_br, dataset_full_labels_br = make_windows_labels_multivariate(df_btc_price_block_reward)

# Obtain the train/test sets of the full windows and labels with block reward included
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(windows=dataset_full_windows_br, labels=dataset_full_labels_br)

# Obtain the train and test datasets
train_dataset, test_dataset = gen_train_test_datasets(X_train=train_windows, y_train=train_labels,
                                                      X_test=test_windows, y_test=test_labels)
print(f"Train dataset: {train_dataset}", f"Test dataset: {test_dataset}")

[*********************100%%**********************]  1 of 1 completed

Head of BTC prices: 
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

              Volume  
Date                  
2014-09-17  21056800  
2014-09-18  34483200  
2014-09-19  37919700  
2014-09-20  36863600  
2014-09-21  26580100  
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3549 entries, 2014-09-17 to 2024-06-04
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3549 non-null   float64
 1   High       3549 non-null   float64
 2   Low       

                 Price
Date                  
2014-09-17  457.334015
2014-09-18  424.440002
2014-09-19  394.795990
2014-09-20  408.903992
2014-09-21  398.821014
Days from block reward 1: -658
 Days from block reward 2: 661
 Days from block reward 3: 2063
 Days from block reward 4: 1485

Head of dataframe with block reward included: 
                  Price block_reward
Date                               
2014-09-17  457.334015           25
2014-09-18  424.440002           25
2014-09-19  394.795990           25
2014-09-20  408.903992           25
2014-09-21  398.821014           25
Tail of dataframe with block reward included: 
                    Price block_reward
Date                                 
2024-05-31  67491.414062         6.25
2024-06-01  67706.937500         6.25
2024-06-02  67751.601562         6.25
2024-06-03  68804.781250         6.25
2024-06-04  70582.593750         6.25
Head of windowed dataframe: 
                  Price block_reward      Price1      Price2      Pri

## Training Prediction

### Multivariate current day pred

In [10]:
df_btc_price_closing = get_historical_btc_data()

# Including block reward to the full BTC closing price dataset
df_btc_price_block_reward = include_block_reward(df_btc_price=df_btc_price_closing)

# Plotting the BTC price and block reward over time
# plot_block_reward_price_data(df_btc_price_block_reward=df_btc_price_block_reward)

# Obtain the full windows and labels with block reward included
dataset_full_windows_br, dataset_full_labels_br = make_windows_labels_multivariate(df_btc_price_block_reward)

# Obtain the train/test sets of the full windows and labels with block reward included
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(windows=dataset_full_windows_br, labels=dataset_full_labels_br)

# Obtain the train and test datasets
train_dataset, test_dataset = gen_train_test_datasets(X_train=train_windows, y_train=train_labels,
                                                      X_test=test_windows, y_test=test_labels)
print(f"Train dataset: {train_dataset}", f"Test dataset: {test_dataset}")

# Create AR model
AR_model = get_AR_model(len(train_windows), WINDOW_SIZE_WEEK + 1)

# Create NBEATS model
NBEATS_model = get_NBEATS_model()

# Create LNRNN model
LNRNN_model = get_LNRNN_model()

# Create LSTM model
LSTM_model_obj = LSTMModel()
LSTM_model = LSTM_model_obj.get_model()

# Create Dense model
dense_model_obj = DenseModel()
dense_model = dense_model_obj.get_model()

train_models = [NBEATS_model, LSTM_model, dense_model]
# train_models = [NBEATS_model, LNRNN_model, LSTM_model, dense_model]
# train_models = [LNRNN_model]

# Obtain list of trained ensemble models
ensemble_models = get_ensemble_models(models=train_models, train_data=train_dataset, test_data=test_dataset,
                                      num_iter=ENSEMBLE_NUM_ITER_TEST, num_epochs=ENSEMBLE_NUM_EPOCHS_TEST)

# Generate model summaries
# get_ensemble_models_summary(models=train_models)

# Plot the ensemble models
# plot_model(AR_model)

# plot_model(NBEATS_model)

# plot_model(LNRNN_model)

# plot_model(LSTM_model)

# plot_model(dense_model)

# Generate ensemble predictions
ensemble_preds = make_ensemble_preds(ensemble_models=ensemble_models, input_data=test_dataset)

# Evaluate ensemble model predictions
ensemble_results = evaluate_preds(y_true=test_labels, y_pred=np.median(ensemble_preds, axis=0))
# print(ensemble_results)

# Obtain the upper and lower bounds of the 95% confidence levels
# lower, upper = get_upper_lower_confidence(preds=ensemble_preds)

# Get the median values of the ensemble preds
ensemble_median = np.median(ensemble_preds, axis=0)

# Plot the confidence interval
# plot_confidence_interval(test_windows, test_labels, ensemble_median, lower=lower, upper=upper, offset=300)

# Make forecasts into future of the price of bitcoin
future_forecast = make_future_forecast(models=ensemble_models, values=dataset_full_labels_br,
                                       into_future=INTO_FUTURE_1_DAY, window_size=WINDOW_SIZE_WEEK + 1)

# plot_future_forecast(df_btc_price=df_btc_price_closing, future_forecast=future_forecast)
print(f"Future forecast: {future_forecast}")

[*********************100%%**********************]  1 of 1 completed

Head of BTC prices: 
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

              Volume  
Date                  
2014-09-17  21056800  
2014-09-18  34483200  
2014-09-19  37919700  
2014-09-20  36863600  
2014-09-21  26580100  
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3549 entries, 2014-09-17 to 2024-06-04
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3549 non-null   float64
 1   High       3549 non-null   float64
 2   Low       


Head of windowed dataframe: 
                  Price block_reward      Price1      Price2      Price3  \
Date                                                                      
2014-09-17  457.334015           25         NaN         NaN         NaN   
2014-09-18  424.440002           25  457.334015         NaN         NaN   
2014-09-19  394.795990           25  424.440002  457.334015         NaN   
2014-09-20  408.903992           25  394.795990  424.440002  457.334015   
2014-09-21  398.821014           25  408.903992  394.795990  424.440002   
2014-09-22  402.152008           25  398.821014  408.903992  394.795990   
2014-09-23  435.790985           25  402.152008  398.821014  408.903992   
2014-09-24  423.204987           25  435.790985  402.152008  398.821014   
2014-09-25  411.574005           25  423.204987  435.790985  402.152008   
2014-09-26  404.424988           25  411.574005  423.204987  435.790985   

                Price4      Price5      Price6      Price7  
Date   

Optimizing model by reducing: mae for epochs: 1, num_iter: 0, model: NBEATS_model
3/3 - 46s - loss: 87351.2734 - mae: 87351.2734 - mse: 21536237568.0000 - val_loss: 1753.1476 - val_mae: 1753.1476 - val_mse: 5880621.0000 - lr: 0.0010 - 46s/epoch - 15s/step
Optimizing model by reducing: mse for epochs: 1, num_iter: 0, model: NBEATS_model
3/3 - 43s - loss: 6412093440.0000 - mae: 46684.5625 - mse: 6412093440.0000 - val_loss: 432320160.0000 - val_mae: 18626.6621 - val_mse: 432320160.0000 - lr: 0.0010 - 43s/epoch - 14s/step
Optimizing model by reducing: mape for epochs: 1, num_iter: 0, model: NBEATS_model
3/3 - 45s - loss: 126.6349 - mae: 9284.0078 - mse: 195750528.0000 - val_loss: 72.5001 - val_mae: 23647.2422 - val_mse: 690143552.0000 - lr: 0.0010 - 45s/epoch - 15s/step
Optimizing model by reducing: mae for epochs: 1, num_iter: 0, model: LSTM_model
3/3 - 3s - loss: 11990.2178 - mae: 11990.2178 - mse: 396130944.0000 - val_loss: 30785.6836 - val_mae: 30785.6836 - val_mse: 1160938496.0000 - l

3/3 - 3s - loss: 340592480.0000 - mae: 11109.5684 - mse: 340592480.0000 - val_loss: 1016501696.0000 - val_mae: 28799.4727 - val_mse: 1016501696.0000 - lr: 0.0010 - 3s/epoch - 947ms/step
Optimizing model by reducing: mape for epochs: 1, num_iter: 0, model: LSTM_model


3/3 - 2s - loss: 84.1327 - mae: 10134.0420 - mse: 281156224.0000 - val_loss: 76.8601 - val_mae: 25376.0137 - val_mse: 790620224.0000 - lr: 0.0010 - 2s/epoch - 597ms/step
Optimizing model by reducing: mae for epochs: 1, num_iter: 0, model: dense_model
3/3 - 1s - loss: 5606.3994 - mae: 5606.3994 - mse: 83807976.0000 - val_loss: 12786.1807 - val_mae: 12786.1807 - val_mse: 201744976.0000 - lr: 0.0010 - 908ms/epoch - 303ms/step
Optimizing model by reducing: mse for epochs: 1, num_iter: 0, model: dense_model
3/3 - 1s - loss: 32113788.0000 - mae: 3485.0310 - mse: 32113788.0000 - val_loss: 68723160.0000 - val_mae: 7398.1699 - val_mse: 68723160.0000 - lr: 0.0010 - 827ms/epoch - 276ms/step
Optimizing model by reducing: mape for epochs: 1, num_iter: 0, model: dense_model
3/3 - 1s - loss: 15.9899 - mae: 1309.4910 - mse: 4577603.0000 - val_loss: 2.9316 - val_mae: 1009.2288 - val_mse: 2318228.7500 - lr: 0.0010 - 860ms/epoch - 287ms/step
1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 18ms/step
Predicting on: 
 Date
2024-05-28    68296.218750
2024-05-29    67578.093750
2024-05-30    68364.992188
2024-05-31    67491.414062
2024-06-01    67706.937500
2024-06-02    67751.601562
2024-06-03    68804.781250
2024-06-04    70582.593750
Name: Price, dtype: float32 -> Prediction: 68159.1328125

Future forecast: [68159.13]


### Multivariate full pipeline

In [ ]:
df_btc_price_closing = get_historical_btc_data()

# Including block reward to the full BTC closing price dataset
df_btc_price_block_reward = include_block_reward(df_btc_price=df_btc_price_closing)

# Plotting the BTC price and block reward over time
# plot_block_reward_price_data(df_btc_price_block_reward=df_btc_price_block_reward)

# Obtain the full windows and labels with block reward included
dataset_full_windows_br, dataset_full_labels_br = make_windows_labels_multivariate(df_btc_price_block_reward)

# Obtain the train/test sets of the full windows and labels with block reward included
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(windows=dataset_full_windows_br, labels=dataset_full_labels_br)

# Obtain the train and test datasets
train_dataset, test_dataset = gen_train_test_datasets(X_train=train_windows, y_train=train_labels,
                                                      X_test=test_windows, y_test=test_labels)
print(f"Train dataset: {train_dataset}", f"Test dataset: {test_dataset}")

# Create AR model
AR_model = get_AR_model(len(train_windows), WINDOW_SIZE_WEEK + 1)

# Create NBEATS model
NBEATS_model = get_NBEATS_model()

# Create LNRNN model
LNRNN_model = get_LNRNN_model()

# Create LSTM model
LSTM_model_obj = LSTMModel()
LSTM_model = LSTM_model_obj.get_model()

# Create Dense model
dense_model_obj = DenseModel()
dense_model = dense_model_obj.get_model()

train_models = [NBEATS_model, LSTM_model, dense_model]
# train_models = [NBEATS_model, LNRNN_model, LSTM_model, dense_model]
# train_models = [LNRNN_model]

# Obtain list of trained ensemble models
ensemble_models = get_ensemble_models(models=train_models, train_data=train_dataset, test_data=test_dataset,
                                      num_iter=ENSEMBLE_NUM_ITER, num_epochs=ENSEMBLE_NUM_EPOCHS)

# Generate model summaries
get_ensemble_models_summary(models=train_models)

# Plot the ensemble models
# plot_model(AR_model)

# plot_model(NBEATS_model)

# plot_model(LNRNN_model)

# plot_model(LSTM_model)

# plot_model(dense_model)

# Generate ensemble predictions
ensemble_preds = make_ensemble_preds(ensemble_models=ensemble_models, input_data=test_dataset)

# Evaluate ensemble model predictions
ensemble_results = evaluate_preds(y_true=test_labels, y_pred=np.median(ensemble_preds, axis=0))
print(ensemble_results)

# Obtain the upper and lower bounds of the 95% confidence levels
lower, upper = get_upper_lower_confidence(preds=ensemble_preds)

# Get the median values of the ensemble preds
ensemble_median = np.median(ensemble_preds, axis=0)

# Plot the confidence interval
# plot_confidence_interval(test_windows, test_labels, ensemble_median, lower=lower, upper=upper, offset=300)

# Make forecasts into future of the price of bitcoin
future_forecast = make_future_forecast(models=ensemble_models, values=dataset_full_labels_br,
                                       into_future=INTO_FUTURE_2_WEEK, window_size=WINDOW_SIZE_WEEK + 1)

# plot_future_forecast(df_btc_price=df_btc_price_closing, future_forecast=future_forecast)
print(f"Future forecast: {future_forecast}")

[*********************100%%**********************]  1 of 1 completed


Head of BTC prices: 
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

              Volume  
Date                  
2014-09-17  21056800  
2014-09-18  34483200  
2014-09-19  37919700  
2014-09-20  36863600  
2014-09-21  26580100  
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3549 entries, 2014-09-17 to 2024-06-04
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3549 non-null   float64
 1   High       3549 non-null   float64
 2   Low       

Streaming output truncated to the last 5000 lines.
3/3 - 0s - loss: 2.5417 - mae: 353.0638 - mse: 651228.1875 - val_loss: 1.7926 - val_mae: 616.6508 - val_mse: 1020710.0000 - lr: 0.0010 - 31ms/epoch - 10ms/step
Epoch 110/300
3/3 - 0s - loss: 2.5544 - mae: 353.2340 - mse: 649174.1875 - val_loss: 1.7824 - val_mae: 613.7260 - val_mse: 1018570.1250 - lr: 0.0010 - 33ms/epoch - 11ms/step
Epoch 111/300
3/3 - 0s - loss: 2.5423 - mae: 352.7848 - mse: 650532.3750 - val_loss: 1.7494 - val_mae: 605.6154 - val_mse: 1023215.3750 - lr: 0.0010 - 32ms/epoch - 11ms/step
Epoch 112/300
3/3 - 0s - loss: 2.5628 - mae: 354.4389 - mse: 654066.2500 - val_loss: 1.7962 - val_mae: 622.4111 - val_mse: 1067867.8750 - lr: 0.0010 - 37ms/epoch - 12ms/step
Epoch 113/300
3/3 - 0s - loss: 2.5851 - mae: 356.7438 - mse: 663637.6250 - val_loss: 1.7586 - val_mae: 609.7024 - val_mse: 1036285.1250 - lr: 0.0010 - 45ms/epoch - 15ms/step
Epoch 114/300
3/3 - 0s - loss: 2.5492 - mae: 354.6795 - mse: 658011.7500 - val_loss: 1.7865 -

### Multivariate train pred

In [ ]:
# Create AR model
AR_model = get_AR_model(len(train_windows), WINDOW_SIZE_WEEK + 1)

# Create NBEATS model
NBEATS_model = get_NBEATS_model()

# Create LNRNN model
LNRNN_model = get_LNRNN_model()

# Create LSTM model
LSTM_model_obj = LSTMModel()
LSTM_model = LSTM_model_obj.get_model()

# Create Dense model
dense_model_obj = DenseModel()
dense_model = dense_model_obj.get_model()

train_models = [NBEATS_model, LSTM_model, dense_model]
# train_models = [NBEATS_model, LNRNN_model, LSTM_model, dense_model]
# train_models = [LNRNN_model]

# Obtain list of trained ensemble models
ensemble_models = get_ensemble_models(models=train_models, train_data=train_dataset, test_data=test_dataset,
                                      num_iter=ENSEMBLE_NUM_ITER, num_epochs=ENSEMBLE_NUM_EPOCHS)

# Generate model summaries
get_ensemble_models_summary(models=train_models)

# Plot the ensemble models
# plot_model(AR_model)

# plot_model(NBEATS_model)

# plot_model(LNRNN_model)

# plot_model(LSTM_model)

# plot_model(dense_model)

# Generate ensemble predictions
ensemble_preds = make_ensemble_preds(ensemble_models=ensemble_models, input_data=test_dataset)

# Evaluate ensemble model predictions
ensemble_results = evaluate_preds(y_true=test_labels, y_pred=np.median(ensemble_preds, axis=0))
print(ensemble_results)

# Obtain the upper and lower bounds of the 95% confidence levels
lower, upper = get_upper_lower_confidence(preds=ensemble_preds)

# Get the median values of the ensemble preds
ensemble_median = np.median(ensemble_preds, axis=0)

# Plot the confidence interval
# plot_confidence_interval(test_windows, test_labels, ensemble_median, lower=lower, upper=upper, offset=300)

# Make forecasts into future of the price of bitcoin
future_forecast = make_future_forecast(models=ensemble_models, values=dataset_full_labels_br,
                                       into_future=INTO_FUTURE_2_WEEK, window_size=WINDOW_SIZE_WEEK + 1)

# plot_future_forecast(df_btc_price=df_btc_price_closing, future_forecast=future_forecast)
print(f"Future forecast: {future_forecast}")